In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(palette = "Dark2")
my_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
  (0.8509803921568627, 0.37254901960784315, 0.00784313725490196)]
pd.set_option('display.max_columns', None)
from itertools import chain, combinations

In [2]:
import importlib
import Model_functions
importlib.reload(Model_functions)
from Model_functions import Model_class 

In [3]:
df = pd.read_csv("df_127_columns.csv")
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,mode_item_id,mode_size,mode_brand_id,mode_item_color,delivery_time_item_id_mean,delivery_time_item_id_std,delivery_time_item_id_min,delivery_time_item_id_max,delivery_time_item_id_median,delivery_time_item_id_mad,delivery_time_size_mean,delivery_time_size_std,delivery_time_size_min,delivery_time_size_max,delivery_time_size_median,delivery_time_size_mad,delivery_time_brand_id_mean,delivery_time_brand_id_std,delivery_time_brand_id_min,delivery_time_brand_id_max,delivery_time_brand_id_median,delivery_time_brand_id_mad,delivery_time_item_color_mean,delivery_time_item_color_std,delivery_time_item_color_min,delivery_time_item_color_max,delivery_time_item_color_median,delivery_time_item_color_mad,delivery_time_user_id_mean,delivery_time_user_id_std,delivery_time_user_id_min,delivery_time_user_id_max,delivery_time_user_id_median,delivery_time_user_id_mad
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.90,0.83,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,55.23,13.65,24.9,79.90,282536.19,5116,59.90,11.66,-0.43,-30.0,0.0,-7.25,-69.90,20.00,-14.67,-45.00,10.0,71,s,20,white,5.04,5.80,1,30,3.0,3.61,10.38,16.45,0,71,4.0,11.27,4.21,5.15,0,71,3.0,2.65,9.80,15.10,0,71,4.0,10.38,3.64,3.38,1,13,3.0,2.20
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.0

In [4]:
df["order_date"] = pd.to_datetime(df["order_date"])

In [ ]:
# How to shift the date
# df_group[['A', 'B', 'C']] = df_gr.groupby('Name')['A', 'B', 'C'].shift(1)

In [5]:
df_group = df[["order_date","user_id"]].drop_duplicates().sort_values(by=["user_id","order_date"])
df_group["order_date_shifted"] = df_group.groupby("user_id")["order_date"].shift(1)
# df_group[['A', 'B', 'C']] = df_gr.groupby('Name')['A', 'B', 'C'].shift(1)
df_group["days_since_last_order"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
# df_group.fillna(0, inplace=True)
df_group

,order_date,user_id,order_date_shifted,days_since_last_order
24723,2012-04-19,6,NaT,NaN
31011,2012-04-25,6,2012-04-19,6.0
226242,2012-09-23,6,2012-04-25,151.0
240276,2012-10-06,6,2012-09-23,13.0
331324,2013-01-11,6,2012-10-06,97.0
...,...,...,...,...
531141,2013-04-29,91911,NaT,NaN
531149,2013-04-27,91912,NaT,NaN
531154,2013-04-29,91915,NaT,NaN
531158,2013-04-29,91920,NaT,NaN


In [7]:
cols_to_agg = ["user_id"]
stats_to_agg = ["mean","median","max","min","std","mad"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group_group = df_group[[i,"days_since_last_order"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group_group.columns = [str("_" + i +"_").join(col) for col in df_group_group.columns]
    df_group_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    # df_group_group.fillna(0,inplace=True)
    df_group = df_group.merge(df_group_group,on=i,how="left")
df_group.fillna(0,inplace=True)
df_group.drop(["order_date_shifted"],axis=1,inplace=True)
df = df.merge(df_group,on=["user_id","order_date"],how="left")    

In [8]:
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,mode_item_id,mode_size,mode_brand_id,mode_item_color,delivery_time_item_id_mean,delivery_time_item_id_std,delivery_time_item_id_min,delivery_time_item_id_max,delivery_time_item_id_median,delivery_time_item_id_mad,delivery_time_size_mean,delivery_time_size_std,delivery_time_size_min,delivery_time_size_max,delivery_time_size_median,delivery_time_size_mad,delivery_time_brand_id_mean,delivery_time_brand_id_std,delivery_time_brand_id_min,delivery_time_brand_id_max,delivery_time_brand_id_median,delivery_time_brand_id_mad,delivery_time_item_color_mean,delivery_time_item_color_std,delivery_time_item_color_min,delivery_time_item_color_max,delivery_time_item_color_median,delivery_time_item_color_mad,delivery_time_user_id_mean,delivery_time_user_id_std,delivery_time_user_id_min,delivery_time_user_id_max,delivery_time_user_id_median,delivery_time_user_id_mad,days_since_last_order,days_since_last_order_user_id_mean,days_since_last_order_user_id_median,days_since_last_order_user_id_max,days_since_last_order_user_id_min,days_since_last_order_user_id_std,days_since_last_order_user_id_mad
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.90,0.83,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,55.23,13.65,24.9,79.90,282536.19,5116,59.90,11.66,-0.43,-30.0,0.0,-7.25,-69.90,20.00,-14.67,-45.00,10.0,71,s,20,white,5.04,5.80,1,30,3.0,3.61,10.38,16.45,0,71,4.0,11.27,4.21,5.15,0,71,3.0,2.65,9.80,15.10,0,71,4.0,10.38,3.64,3.38,1,13,3.0,2.20,0.0,66.6,46.0,137.0,15.0,58.22,49.52
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,

In [19]:
df_group = df[["order_date","user_id","size"]].drop_duplicates().sort_values(by=["user_id","order_date"])
df_group["order_date_shifted"] = df_group.groupby(["user_id","size"])["order_date"].shift(1)
# df_group["days_since_last_order"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
# df_group.fillna(0, inplace=True)
df_group["days_since_last_order"+ "_size"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
df_group[df_group["user_id"]== 6] 

,order_date,user_id,size,order_date_shifted,days_since_last_order_size
24723,2012-04-19,6,s,NaT,NaN
31011,2012-04-25,6,s,2012-04-19,6.0
226242,2012-09-23,6,s,2012-04-25,151.0
226244,2012-09-23,6,xs,NaT,NaN
240276,2012-10-06,6,s,2012-09-23,13.0
240277,2012-10-06,6,xs,2012-09-23,13.0
240280,2012-10-06,6,m,NaT,NaN
240282,2012-10-06,6,unsized,NaT,NaN
331324,2013-01-11,6,s,2012-10-06,97.0
331329,2013-01-11,6,unsized,2012-10-06,97.0


In [ ]:
df_group = df[["order_date","user_id","size"]].drop_duplicates().sort_values(by=["user_id","order_date"])
df_group["order_date_shifted"] = df_group.groupby(["user_id","size"])["order_date"].shift(1)
# df_group["days_since_last_order"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
# df_group.fillna(0, inplace=True)
df_group["days_since_last_order"+ "_size"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
df_group[df_group["user_id"]== 6] 

In [20]:
item_descriptions = ["item_id","size","item_color","brand_id"]
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

In [24]:
categorical_powerset = powerset(item_descriptions)
# Use the powerset to create new columns indicating the number of the same items in the order
powerset_name_list = []
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    current = list(column)
    if not all(x in current for x in exclude):
        print(current)
        df_group = df[["order_date","user_id"]+current].drop_duplicates().sort_values(by=["user_id","order_date"])
        df_group["order_date_shifted"] = df_group.groupby(["user_id"] + current)["order_date"].shift(1)
        df_group["days_since_last_order_same"+ "_" + "_".join(current)] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
        df_group.fillna(0, inplace=True)
        df_group.drop(["order_date_shifted"],axis=1,inplace=True)
        df = df.merge(df_group,on=["user_id","order_date"]+current,how="left")
        
        # name = "order_number_same_" + "_".join(current)
        # name_different = "order_number_different_" + "_".join(current)
        # powerset_name_list += [name]
        # df_group = df[current + ["order_id","user_title"]].groupby(["order_id"] + current).count().rename(columns={"user_title":name})
        # df = pd.merge(df, df_group, on = current + ["order_id"], how = "left")
        # # Get number of different items in the order
        # df[name_different] = df["order_item_count"] - df[name] 
        # # df[name] = df[name] - 1

['item_id']
['size']
['item_color']
['brand_id']
['item_id', 'size']
['item_id', 'item_color']
['size', 'item_color']
['size', 'brand_id']
['item_color', 'brand_id']
['item_id', 'size', 'item_color']
['size', 'item_color', 'brand_id']


In [23]:
# test = ["user_id", "item_id",]
# a = "days_since_last_order_same"
# # join a and tes list as strings
# b = "_".join(test)
# # join a and b as strings
# c = a + "_" + b
# c

'days_since_last_order_same_user_id_item_id'

In [25]:
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,mode_item_id,mode_size,mode_brand_id,mode_item_color,delivery_time_item_id_mean,delivery_time_item_id_std,delivery_time_item_id_min,delivery_time_item_id_max,delivery_time_item_id_median,delivery_time_item_id_mad,delivery_time_size_mean,delivery_time_size_std,delivery_time_size_min,delivery_time_size_max,delivery_time_size_median,delivery_time_size_mad,delivery_time_brand_id_mean,delivery_time_brand_id_std,delivery_time_brand_id_min,delivery_time_brand_id_max,delivery_time_brand_id_median,delivery_time_brand_id_mad,delivery_time_item_color_mean,delivery_time_item_color_std,delivery_time_item_color_min,delivery_time_item_color_max,delivery_time_item_color_median,delivery_time_item_color_mad,delivery_time_user_id_mean,delivery_time_user_id_std,delivery_time_user_id_min,delivery_time_user_id_max,delivery_time_user_id_median,delivery_time_user_id_mad,days_since_last_order,days_since_last_order_user_id_mean,days_since_last_order_user_id_median,days_since_last_order_user_id_max,days_since_last_order_user_id_min,days_since_last_order_user_id_std,days_since_last_order_user_id_mad,days_since_last_order_same_item_id,days_since_last_order_same_size,days_since_last_order_same_item_color,days_since_last_order_same_brand_id,days_since_last_order_same_item_id_size,days_since_last_order_same_item_id_item_color,days_since_last_order_same_size_item_color,days_since_last_order_same_size_brand_id,days_since_last_order_same_item_color_brand_id,days_since_last_order_same_item_id_size_item_color,days_since_last_order_same_size_item_color_brand_id
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.9

In [ ]:
low_cardinality_cat_cols = ["size","item_color","user_title","user_state"]
high_cardinality_cat_cols = ["item_id","brand_id","user_id"]
cat_cols = low_cardinality_cat_cols + high_cardinality_cat_cols
cat_cols += ["mode_item_id","mode_size","mode_brand_id","mode_item_color",]

In [32]:
# Instantiate the model class
Model = Model_class(df)

In [ ]:
# Split the data into train and test sets
df_train,df_test, df_valid = Model.split_data(False)
# Encode the categorical variables using leave-one-out encoding
df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.1,True)